In [3]:
!pip install psycopg2-binary


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 883.0 kB/s eta 0:00:00a 0:00:01


In [5]:
import pandas as pd
import psycopg2

In [6]:
# DB connection setup using hardcoded credentials (for onboarding only)
conn = psycopg2.connect(
    dbname="neondb",
    user="neondb_owner",
    password="npg_CeS9fJg2azZD",
    host="ep-falling-glitter-a5m0j5gk-pooler.us-east-2.aws.neon.tech",
    port="5432",
    sslmode="require"
)
cur = conn.cursor()

In [8]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine

# 🔑 Replace with your real credentials from day3_sql_combined_with_creds.ipynb
db_user = "your_username"
db_pass = "your_password"
db_host = "localhost"   # or the host in your creds file
db_port = "5432"
db_name = "training"

# SQLAlchemy connection string
engine = create_engine(f"postgresql+psycopg2://{db_user}:{db_pass}@{db_host}:{db_port}/{db_name}")


In [16]:
from sqlalchemy import create_engine
import pandas as pd

# build engine string
db_user = "neondb_owner"
db_pass = "npg_CeS9fJg2azZD"
db_host = "ep-falling-glitter-a5m0j5gk-pooler.us-east-2.aws.neon.tech"
db_port = "5432"
db_name = "neondb"

engine = create_engine(
    f"postgresql+psycopg2://{db_user}:{db_pass}@{db_host}:{db_port}/{db_name}?sslmode=require"
)

# test query
query = "SELECT * FROM nyc_schools.high_school_directory LIMIT 5;"
df = pd.read_sql(query, engine)
df.head()


,dbn,school_name,borough,building_code,phone_number,fax_number,grade_span_min,grade_span_max,expgrade_span_min,expgrade_span_max,...,number_programs,Location 1,Community Board,Council District,Census Tract,Zip Codes,Community Districts,Borough Boundaries,City Council Districts,Police Precincts
0,27Q260,Frederick Douglass Academy VI High School,Queens,Q465,718-471-2154,718-471-2890,9.0,12,None,None,...,1,"{'latitude': '40.601989336', 'longitude': '-73...",14,31,100802,20529,51,3,47,59
1,21K559,Life Academy High School for Film and Music,Brooklyn,K400,718-333-7750,718-333-7775,9.0,12,None,None,...,1,"{'latitude': '40.593593811', 'longitude': '-73...",13,47,306,17616,21,2,45,35
2,16K393,Frederick Douglass Academy IV Secondary School,Brooklyn,K026,718-574-2820,718-574-2821,9.0,12,None,None,...,1,"{'latitude': '40.692133704', 'longitude': '-73...",3,36,291,18181,69,2,49,52
3,08X305,Pablo Neruda Academy,Bronx,X450,718-824-1682,718-824-1663,9.0,12,None,None,...,1,"{'latitude': '40.822303765', 'longitude': '-73...",9,18,16,11611,58,5,31,26
4,03M485,Fiorello H. LaGuardia High School of Music & A...,Manhattan,M485,212-496-0700,212-724-5748,9.0,12,None,None,...,6,"{'latitude': '40.773670507', 'longitude': '-73...",7,6,151,12420,20,4,19,12


School Distribution (number of schools per borough)

In [17]:
query = """
SELECT borough, COUNT(DISTINCT dbn) AS total_schools
FROM nyc_schools.high_school_directory
GROUP BY borough
ORDER BY total_schools DESC;
"""
df_schools = pd.read_sql(query, engine)
df_schools


,borough,total_schools
0,Brooklyn,121
1,Bronx,118
2,Manhattan,106
3,Queens,80
4,Staten Island,10


Insight: Brooklyn has the most schools (121), while Staten Island has the least (10).

Average % of English Language Learners (ELL) per borough

In [20]:
query = """
SELECT h.borough, ROUND(AVG(d.ell_percent)::numeric, 2) AS avg_ell_percent
FROM nyc_schools.school_demographics d
JOIN nyc_schools.high_school_directory h
    ON d.dbn = h.dbn
GROUP BY h.borough
ORDER BY avg_ell_percent DESC;
"""
df_ell = pd.read_sql(query, engine)
df_ell


,borough,avg_ell_percent
0,Manhattan,7.57


Insight: Manhattan has an average of 7.6% English Language Learners (ELL), which suggests a relatively low concentration of ELL students compared to other boroughs. This may reflect different demographic patterns and language diversity across the city.

In [22]:
query = """
SELECT h.borough, h.school_name, d.sped_percent
FROM nyc_schools.school_demographics d
JOIN nyc_schools.high_school_directory h
    ON d.dbn = h.dbn
WHERE d.sped_percent IS NOT NULL
ORDER BY h.borough, d.sped_percent DESC
LIMIT 15;  -- 3 per borough × 5 boroughs
"""
df_sped = pd.read_sql(query, engine)
df_sped


,borough,school_name,sped_percent
0,Manhattan,East Side Community School,28.8
1,Manhattan,East Side Community School,27.7
2,Manhattan,East Side Community School,26.7
3,Manhattan,East Side Community School,26.4
4,Manhattan,Marta Valle High School,25.9
5,Manhattan,East Side Community School,25.1
6,Manhattan,Henry Street School for International Studies,25.1
7,Manhattan,Henry Street School for International Studies,24.9
8,Manhattan,East Side Community School,24.8
9,Manhattan,East Side Community School,24.5


Top 3 Schools per Borough by Special Education %

In [23]:
query = """
WITH ranked_schools AS (
    SELECT 
        h.borough,
        h.school_name,
        d.sped_percent,
        ROW_NUMBER() OVER (
            PARTITION BY h.borough
            ORDER BY d.sped_percent DESC
        ) AS rank
    FROM nyc_schools.school_demographics d
    JOIN nyc_schools.high_school_directory h
        ON d.dbn = h.dbn
    WHERE d.sped_percent IS NOT NULL
)
SELECT borough, school_name, sped_percent
FROM ranked_schools
WHERE rank <= 3
ORDER BY borough, rank;
"""
df_sped = pd.read_sql(query, engine)
df_sped


,borough,school_name,sped_percent
0,Manhattan,East Side Community School,28.8
1,Manhattan,East Side Community School,27.7
2,Manhattan,East Side Community School,26.7


The East Side Community School in Manhattan consistently shows the highest percentages of special education students (26–29%), highlighting its strong role in supporting students with special needs.

##  Key Insights from NYC School Demographics
School Distribution: Brooklyn has the highest number of schools (121), followed closely by the Bronx (118) and Manhattan (106). Staten Island has the fewest with only 10 schools, showing an uneven distribution across boroughs.

English Language Learners (ELL): Manhattan reports the lowest average percentage of English Language Learners at 7.57%, suggesting that language support needs are less concentrated there compared to other boroughs.

Special Education Support: The top schools providing the most support for special education students are located in Manhattan, with East Side Community School appearing as a consistent leader, highlighting its strong focus on inclusive education.

The data highlights that school availability is uneven across boroughs, language support needs are relatively lower in Manhattan, and specialized support for students with disabilities is concentrated in a few standout schools like East Side Community School.